In [1]:
# permite a utilização de caminhos relativos ao projeto, mesmo no diretório de notebooks

from knapsax.utils import setrootdir

setrootdir("knapsax")

'Directory knapsax successfully loaded as current working directory.'

In [2]:
from typing import List, Tuple, Dict, Callable
import random

import numpy as np

from knapsax.optimization import Problem, Maximize, Solution, Knapsack

In [3]:
knapsack = Knapsack(instance_file="data/knapsack-instance.txt")
knapsack

Knapsack(file=data/knapsack-instance.txt, n_items=100, capacity=1550)

In [4]:
class ACOSolution(Solution):
    def __init__(self, knapsack, pheromone_vector, alpha=1, beta=1):
        super().__init__(knapsack)
        self.pheromone_vector = pheromone_vector
        self.alpha = alpha
        self.beta = beta

    def generate_solution(self):
        available_items = set(range(self.knapsack.n_items))

        while available_items:
            probabilities = self._compute_probabilities(available_items)

            if probabilities.sum() == 0 or len(available_items) == 0:
                break

            item_index = np.random.choice(list(available_items), p=probabilities)

            self.add_item(item_index)
            available_items.remove(item_index)

    def _compute_probabilities(self, available_items):
        pheromone = np.array([self.pheromone_vector[i] for i in available_items])
        heuristic = np.array([
            self.knapsack.items[i].value / self.knapsack.items[i].weight for i in available_items
        ])
        scores = (pheromone ** self.alpha) * (heuristic ** self.beta)
        total = scores.sum()
        if total == 0:
            return np.ones_like(scores) / len(scores)
        return scores / total


In [5]:
class ACO:
    def __init__(self, knapsack, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        self.knapsack = knapsack
        self.n_items = knapsack.n_items
        self.pheromone = np.ones(self.n_items) / self.n_items
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        best_solution = None

        for _ in range(self.n_iterations):
            all_solutions = self.gen_all_solutions()
            self.spread_pheromone(all_solutions)

            local_best = max(all_solutions, key=lambda s: s.total_value)
            if best_solution is None or local_best.total_value > best_solution.total_value:
                best_solution = local_best

            self.pheromone *= self.decay

        return best_solution.items, best_solution.total_value

    def spread_pheromone(self, all_solutions):
        sorted_solutions = sorted(all_solutions, key=lambda s: s.total_value, reverse=True)
        for solution in sorted_solutions[:self.n_best]:
            for idx in solution.items:
                self.pheromone[idx] += solution.total_value / sum(item.value for item in self.knapsack.items)

    def gen_all_solutions(self):
        return [self.gen_solution() for _ in range(self.n_ants)]

    def gen_solution(self):
        solution = ACOSolution(self.knapsack, self.pheromone, self.alpha, self.beta)
        solution.generate_solution()
        return solution

class Solution:
    def __init__(self, knapsack):
        self.knapsack = knapsack
        self.items = []
        self.total_weight = 0
        self.total_value = 0

    def add_item(self, item_index):
        item = self.knapsack.items[item_index]
        if self.total_weight + item.weight <= self.knapsack.capacity:
            self.items.append(item_index)
            self.total_weight += item.weight
            self.total_value += item.value
            return True
        return False

    def __repr__(self):
        return (f'Solution(value={self.total_value}, weight={self.total_weight}, '
                f'items={self.items})')

In [6]:
aco = ACO(
    knapsack,
    n_ants=50, n_best=10, n_iterations=100,
    decay=0.95, alpha=1, beta=2
)
best_solution, best_value = aco.run()

print("Best Value:", best_value)
print("Items:", [knapsack.items[i] for i in best_solution])

Best Value: 2190
Items: [Item(value=33, weight=23), Item(value=38, weight=28), Item(value=27, weight=19), Item(value=16, weight=10), Item(value=19, weight=13), Item(value=17, weight=11), Item(value=32, weight=21), Item(value=34, weight=22), Item(value=45, weight=29), Item(value=53, weight=38), Item(value=29, weight=21), Item(value=44, weight=28), Item(value=23, weight=15), Item(value=26, weight=18), Item(value=52, weight=37), Item(value=43, weight=31), Item(value=44, weight=33), Item(value=37, weight=24), Item(value=36, weight=25), Item(value=48, weight=32), Item(value=64, weight=48), Item(value=56, weight=41), Item(value=28, weight=20), Item(value=63, weight=47), Item(value=58, weight=43), Item(value=39, weight=25), Item(value=24, weight=17), Item(value=55, weight=40), Item(value=49, weight=34), Item(value=81, weight=58), Item(value=63, weight=47), Item(value=46, weight=30), Item(value=51, weight=37), Item(value=57, weight=42), Item(value=50, weight=35), Item(value=40, weight=27), Ite